In [1]:
import pandas as pd
import numpy as np
import string as str
import pandas_profiling as pp
import sklearn.metrics as met

In [104]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")


In [3]:
df_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
print(df_train.shape)
print(df_test.shape)

(614, 13)
(367, 12)


In [105]:
df_train["src"] = "train"
df_test["src"] = "test"

In [106]:
df = pd.concat([df_train, df_test], axis=0, sort=False)

In [7]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,src
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,train
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,train
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,train
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,train
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,train


In [119]:
pp.ProfileReport(df)

Number of variables,27
Number of observations,925
Total Missing (%),1.3%
Total size in memory,195.2 KiB
Average record size in memory,216.1 B
Numeric,6
Categorical,8
Boolean,12
Date,0
Text (Unique),1
Rejected,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 366
Data columns (total 14 columns):
Loan_ID              981 non-null object
Gender               957 non-null object
Married              978 non-null object
Dependents           956 non-null object
Education            981 non-null object
Self_Employed        926 non-null object
ApplicantIncome      981 non-null int64
CoapplicantIncome    981 non-null float64
LoanAmount           954 non-null float64
Loan_Amount_Term     961 non-null float64
Credit_History       902 non-null float64
Property_Area        981 non-null object
Loan_Status          614 non-null object
src                  981 non-null object
dtypes: float64(4), int64(1), object(9)
memory usage: 115.0+ KB


In [130]:
df.isna().sum()

Loan_ID                   0
Gender                    0
Married                   0
Dependents                0
Education                 0
Self_Employed             0
ApplicantIncome           0
CoapplicantIncome         0
LoanAmount                0
Loan_Amount_Term          0
Credit_History            0
Property_Area             0
Loan_Status             333
src                       0
Total_Income              0
Is_Male                   0
Is_Female                 0
Is_Married                0
Is_Not_Married            0
Is_Graduate               0
Is_Not_Graduate           0
Is_Self_Employed          0
Is_Not_Self_Employed      0
Is_PA_Semiurban           0
Is_PA_Urban               0
Is_PA_Rural               0
dtype: int64

In [107]:
df["Gender"].fillna(df["Gender"].mode()[0], inplace=True)
df["Married"].fillna(df["Married"].mode()[0], inplace=True)
df["Self_Employed"].fillna(df["Self_Employed"].mode()[0], inplace=True)
df["Dependents"].fillna(df["Dependents"].mode()[0], inplace=True)
df["Loan_Amount_Term"].fillna(df["Loan_Amount_Term"].mode()[0], inplace=True)

In [108]:
df.loc[(df.Loan_Status == "Y") & (df.Credit_History.isna()), "Credit_History"] = 1
df.loc[(df.Loan_Status == "N") & (df.Credit_History.isna()), "Credit_History"] = 0

In [109]:
df["Total_Income"] = df.ApplicantIncome + df.CoapplicantIncome

In [110]:
df.loc[df["Gender"] == "Male", "Is_Male"] = 1
df.loc[df["Gender"] == "Female", "Is_Male"] = 0

df.loc[df["Gender"] == "Female", "Is_Female"] = 1
df.loc[df["Gender"] == "Male", "Is_Female"] = 0

In [111]:
df.loc[df["Married"] == "Yes", "Is_Married"] = 1
df.loc[df["Married"] == "No", "Is_Married"] = 0

df.loc[df["Married"] == "No", "Is_Not_Married"] = 1
df.loc[df["Married"] == "Yes", "Is_Not_Married"] = 0

In [112]:
df.loc[df["Education"] == "Graduate", "Is_Graduate"] = 1
df.loc[df["Education"] == "Not Graduate", "Is_Graduate"] = 0

df.loc[df["Education"] == "Not Graduate", "Is_Not_Graduate"] = 1
df.loc[df["Education"] == "Graduate", "Is_Not_Graduate"] = 0

In [113]:
df.loc[df["Self_Employed"] == "Yes", "Is_Self_Employed"] = 1
df.loc[df["Self_Employed"] == "No", "Is_Self_Employed"] = 0

df.loc[df["Self_Employed"] == "No", "Is_Not_Self_Employed"] = 1
df.loc[df["Self_Employed"] == "Yes", "Is_Not_Self_Employed"] = 0

In [47]:
df.Dependents.value_counts()

0     554
2     157
1     154
3+     89
Name: Dependents, dtype: int64

In [114]:
df.loc[df.Dependents == "3+"]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,...,src,Total_Income,Is_Male,Is_Female,Is_Married,Is_Not_Married,Is_Graduate,Is_Not_Graduate,Is_Self_Employed,Is_Not_Self_Employed
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,...,train,5540.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
34,LP001100,Male,No,3+,Graduate,No,12500,3000.0,320.0,360.0,...,train,15500.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
61,LP001206,Male,Yes,3+,Graduate,No,3029,0.0,99.0,360.0,...,train,3029.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
68,LP001238,Male,Yes,3+,Not Graduate,Yes,7100,0.0,125.0,60.0,...,train,7100.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
73,LP001250,Male,Yes,3+,Not Graduate,No,4755,0.0,95.0,360.0,...,train,4755.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
74,LP001253,Male,Yes,3+,Graduate,Yes,5266,1774.0,187.0,360.0,...,train,7040.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
78,LP001263,Male,Yes,3+,Graduate,No,3167,4000.0,180.0,300.0,...,train,7167.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
79,LP001264,Male,Yes,3+,Not Graduate,Yes,3333,2166.0,130.0,360.0,...,train,5499.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
109,LP001384,Male,Yes,3+,Not Graduate,No,2071,754.0,94.0,480.0,...,train,2825.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
126,LP001448,Male,Yes,3+,Graduate,No,23803,0.0,370.0,360.0,...,train,23803.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [115]:
df.loc[df.Dependents == "3+", "Dependents"] = 5

In [116]:
df.loc[df["Property_Area"] == "Semiurban", "Is_PA_Semiurban"] = 1
df.loc[df["Property_Area"].isin(["Urban", "Rural"]), "Is_PA_Semiurban"] = 0

df.loc[df["Property_Area"] == "Urban", "Is_PA_Urban"] = 1
df.loc[df["Property_Area"].isin(["Semiurban", "Rural"]), "Is_PA_Urban"] = 0

df.loc[df["Property_Area"] == "Rural", "Is_PA_Rural"] = 1
df.loc[df["Property_Area"].isin(["Semiurban", "Urban"]), "Is_PA_Rural"] = 0

In [137]:
df.Dependents.value_counts()

0.0    537
2.0    153
1.0    151
5.0     84
Name: Dependents, dtype: int64

In [117]:
df.dropna(subset=['LoanAmount'], inplace=True)
df.dropna(subset=['Credit_History'], inplace=True)

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 925 entries, 1 to 366
Data columns (total 26 columns):
Loan_ID                 925 non-null object
Gender                  925 non-null object
Married                 925 non-null object
Dependents              925 non-null float64
Education               925 non-null object
Self_Employed           925 non-null object
ApplicantIncome         925 non-null int64
CoapplicantIncome       925 non-null float64
LoanAmount              925 non-null float64
Loan_Amount_Term        925 non-null float64
Credit_History          925 non-null float64
Property_Area           925 non-null object
Loan_Status             592 non-null object
src                     925 non-null object
Total_Income            925 non-null float64
Is_Male                 925 non-null float64
Is_Female               925 non-null float64
Is_Married              925 non-null float64
Is_Not_Married          925 non-null float64
Is_Graduate             925 non-null float64
Is_No

In [134]:
xcols = ["Dependents", "LoanAmount", "Loan_Amount_Term", "Credit_History", "Total_Income",
         "Is_Male", "Is_Female", "Is_Married", "Is_Not_Married", "Is_Graduate", "Is_Not_Graduate", "Is_Self_Employed", 
         "Is_Not_Self_Employed", "Is_PA_Semiurban", "Is_PA_Urban", "Is_PA_Rural"]
ycol = ["Loan_Status"]

dfx_train = df.loc[df.src == "train", xcols]

In [136]:
dfx_train.head()

,Dependents,LoanAmount,Loan_Amount_Term,Credit_History,Total_Income,Is_Male,Is_Female,Is_Married,Is_Not_Married,Is_Graduate,Is_Not_Graduate,Is_Self_Employed,Is_Not_Self_Employed,Is_PA_Semiurban,Is_PA_Urban,Is_PA_Rural
1,1.0,128.0,360.0,1.0,6091.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,66.0,360.0,1.0,3000.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,120.0,360.0,1.0,4941.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.0,141.0,360.0,1.0,6000.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
5,2.0,267.0,360.0,1.0,9613.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
